In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext

In [3]:
# conf = SparkConf()
# conf.set("spark.master","local[2]")
# conf.set("spark.app.name","sparkapp")
# sc = SparkContext(conf = conf)
# sc

In [4]:
# ssc = StreamingContext(sparkContext=sc,batchDuration=20)
# ssc

In [6]:
#windowing in spark SQL
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)
sqlc

In [16]:
d1 = sqlc.createDataFrame([(1,2300,1),(2,3000,1),(1,1000,2),(2,1500,2),(2,4200,3),(3,2000,1),(2,500,4),(3,2000,2)],["id","sales","month"])
d1.show()
d1.createOrReplaceTempView("t1")
# sqlc.sql("select id,max(sales),min(sales) from t1 group by id").show()
'''
2300-->2300
1000-->2300+1000/2=1650
2000-->2300+1000+2000/3=1780

For every month, what is the average sale
'''
sqlc.sql("Select id,sales,month,avg(sales) over(partition by id order by month) as avgsale from t1").show()

+---+-----+-----+
| id|sales|month|
+---+-----+-----+
|  1| 2300|    1|
|  2| 3000|    1|
|  1| 1000|    2|
|  2| 1500|    2|
|  2| 4200|    3|
|  3| 2000|    1|
|  2|  500|    4|
|  3| 2000|    2|
+---+-----+-----+

+---+-----+-----+-------+
| id|sales|month|avgsale|
+---+-----+-----+-------+
|  1| 2300|    1| 2300.0|
|  1| 1000|    2| 1650.0|
|  3| 2000|    1| 2000.0|
|  3| 2000|    2| 2000.0|
|  2| 3000|    1| 3000.0|
|  2| 1500|    2| 2250.0|
|  2| 4200|    3| 2900.0|
|  2|  500|    4| 2300.0|
+---+-----+-----+-------+



In [ ]:
'''
score

for every over, i need agg (avg,total) of the runs 
1 over-->3 -->3
2 over-->4-->3.5
3 over-->2-->3
4 over -->5-->3.4

for every , 
'''

In [33]:
ssc = StreamingContext(sparkContext=sc,batchDuration=20)
ssc

In [34]:
dstream1 = ssc.socketTextStream("localhost",12345)
dstream2 = dstream1.window(windowDuration=60)
'''
windowduration must be always a multiple of batch duration
number of multiples of batch duration define window size
'''
dstream3 = dstream2.flatMap(lambda x:x.split(" ")).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
dstream3.pprint()

In [37]:
# ssc.start()

In [36]:
ssc.stop(False)